In [1]:
# going to clean the reviews here.
import ast
import sys
import string
import json
import random
import time
import autopep8
import joblib
import gc
import os
import glob
import numpy as np
import pandas as pd
from IPython.display import display
from pandas.api.types import is_numeric_dtype
from scipy.sparse import hstack
from io import StringIO

# do get rid of annoying warnings
pd.set_option('future.no_silent_downcasting', True)

In [6]:
# global, for the print statements
verbose = True
# set to True so that the notebook tries smaller chunks and only does 5 chunks
test = False
# How confident does the model need to be to accept the psuedo-label
threshold = 0.8
# chunk_size (will only use this if test if False)
CHUNK_SIZE = 100000

chunk_save_path = f"files/review_chunks/no_label/review_chunk{'_test' if test else ''}_0"

In [7]:
# this should probably be on a seperate cell so I don't constantly reload the dataframes
rwpath = 'reviews/yelpReviews/yelp_academic_dataset_review.json'

rwsub = ['user_id', 'business_id', 'stars', 'text', 'date']
# for bootstrap model, we will need to link the user_ids to this for
# when training the bigger model. For actual bootstrap training we'll
# stip the user out of it
rwsub_less = ['user_id','business_id','stars', 'text']

In [8]:
# clean and then predict. returns predictions
def clean_predict(in_chunk_df):
    in_chunk_df = in_chunk_df
    # clean text and then normalize rating
    
    return y_pred

# converts to lowercase and strip punctuation
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [9]:
# going to have read the json file in chunks, the thing is almost 5 gigs
chunk_size = 200 if test else CHUNK_SIZE

with open(rwpath, 'r', encoding='utf-8') as file:
        chunk = []
        chunk_df = pd.DataFrame()
        chunk_count = []
        count = 1
        print(f'Starting labeling')
        t0 = time.time()
        # check if this chunk exists already (for restarts)
        chunk_path = f'{chunk_save_path}{count}.csv'
        
        if os.path.exists(chunk_path):
            chunk_count.append(count)
        
        for index, line in enumerate(file):            
            if count not in chunk_count:
                # read each line as a dataframe then append to a list
                data = json.loads(line)
                #data = pd.read_json(StringIO(line), lines = True)
                chunk.append(data)
            
            # check if chunk is full / right now we exit since I'm just trying to clean the thing rn.
            if (index + 1) % chunk_size == 0:
                if os.path.exists(chunk_path):
                    print(f'Chunk {count} already exits at {chunk_path}. Skipping')
                else:
                    print(f'Starting labeling of chunk {count}.')
                    chunk_df = pd.DataFrame(chunk)
                    
                    # remove the columns we don't care about
                    chunk_df = chunk_df[rwsub_less]
        
                    # predict pseudo labels
                    chunk_df['text'] = chunk_df['text'].apply(clean_text)
                    # write each chunk to its own file, will combine them later
                    chunk_df.to_csv(chunk_path, index=False)
        
                    if verbose:
                        print(f'chunk {count} finished at {time.time() - t0} seconds. Saved at {chunk_path}\n')
                        
                # garbage collection
                del chunk, chunk_df
                gc.collect()
        
                chunk = []
                chunk_df = pd.DataFrame()
                
                if count not in chunk_count:
                    chunk_count.append(count)
                count += 1
                chunk_path = f'{chunk_save_path}{count}.csv'
                
                if os.path.exists(chunk_path):
                    chunk_count.append(count)
                
                if test and count > 5:
                    break
if test:
    print('TEST RUN')
print(f'Finished chunking reviews after {(time.time() - t0) / 60.0} minutes. Files are seperated into chunks of {chunk_size} lines.')


Starting labeling
Starting labeling of chunk 1.
chunk 1 finished at 2.133465051651001 seconds. Saved at files/review_chunks/no_label/review_chunk_01.csv

Starting labeling of chunk 2.
chunk 2 finished at 4.29779314994812 seconds. Saved at files/review_chunks/no_label/review_chunk_02.csv

Starting labeling of chunk 3.
chunk 3 finished at 8.058505773544312 seconds. Saved at files/review_chunks/no_label/review_chunk_03.csv

Starting labeling of chunk 4.
chunk 4 finished at 12.565075159072876 seconds. Saved at files/review_chunks/no_label/review_chunk_04.csv

Starting labeling of chunk 5.
chunk 5 finished at 16.918478965759277 seconds. Saved at files/review_chunks/no_label/review_chunk_05.csv

Starting labeling of chunk 6.
chunk 6 finished at 19.347845315933228 seconds. Saved at files/review_chunks/no_label/review_chunk_06.csv

Starting labeling of chunk 7.
chunk 7 finished at 21.691052675247192 seconds. Saved at files/review_chunks/no_label/review_chunk_07.csv

Starting labeling of chunk 